In [1]:
from keras.models import * 
from keras.layers import *
from keras.applications import *
from keras.preprocessing.image import *
from keras.utils.training_utils import multi_gpu_model
import tensorflow as tf

Using TensorFlow backend.


In [2]:
with tf.device('/cpu:0'):
    input_tensor = Input((299, 299, 3))
    x = Lambda(xception.preprocess_input)(input_tensor)

    base_model = Xception(input_tensor=x, weights=None, include_top=False)
    m_out = base_model.output
    p_out = GlobalAveragePooling2D()(m_out)
    p_out = Dropout(1)(p_out)
    predictions = Dense(3, activation='softmax')(p_out)
    model = Model(inputs=base_model.input, outputs=predictions)
    model.load_weights('Xception_finetune_80.h5')

parallel_model = multi_gpu_model(model, gpus=2)
model.summary()

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            (None, 299, 299, 3)  0                                            
__________________________________________________________________________________________________
lambda_1 (Lambda)               (None, 299, 299, 3)  0           input_1[0][0]                    
__________________________________________________________________________________________________
block1_conv1 (Conv2D)           (None, 149, 149, 32) 864         lambda_1[0][0]                   
__________________________________________________________________________________________________
block1_conv1_bn (BatchNormaliza (None, 149, 149, 32) 128         block1_conv1[0][0]               
__________________________________________________________________________________________________
block1_con

In [4]:
batch_size = 128
img_size = (299, 299)

gen = ImageDataGenerator()
test_generator = gen.flow_from_directory("/home/hdd0/Develop/tct/train_c1/stain_algo_compare/alldata_test/", 
                                         img_size, 
                                         shuffle=False,
                                         batch_size=batch_size)
test_img_nums = test_generator.samples
all_test_results = parallel_model.predict_generator(test_generator, (test_img_nums//batch_size) + 1, verbose=1)
all_test_results = all_test_results[:test_img_nums]
all_labels = test_generator.classes

Found 87 images belonging to 3 classes.
1/1 [==============================] - 4s 4s/step


In [5]:
class_label_dict = test_generator.class_indices

def get_key(dict, value):
    return [k for k, v in dict.items() if v == value]

# create class num lens dict, every dict store current class predict num
total_predictions_dict = {}
for row_class_name, label in class_label_dict.items():
    total_predictions_dict[row_class_name] = {}
    for column_class_name, label in class_label_dict.items():
        total_predictions_dict[row_class_name][column_class_name] = 0

In [6]:
for i, label in enumerate(all_labels):
    # get the first result
    label_class_name = get_key(class_label_dict, label)[0]
    test_class_name = get_key(class_label_dict, np.argmax(test[i]))[0]
#     print(label_class_name)
#     print(total_predictions_dict[label_class_name].keys())
    total_predictions_dict[label_class_name][test_class_name] += 1

NameError: name 'test' is not defined

In [ ]:
print(total_predictions_dict['MC'])

In [ ]:
print(total_predictions_dict['LSIL'])

In [ ]:
print(total_predictions_dict['HSIL'])

In [ ]:
import csv
out = open('confusion_matrix.csv','a', newline='')
csv_write = csv.writer(out,dialect='excel')

# write the title
line = [class_name for class_name, label in class_label_dict.items()]
line = [" "] + line + ["TOTAL"]
csv_write.writerow(line)

# write rows

for row_class_name, label in class_label_dict.items():
    one_class_total_predict = 0
    line = [row_class_name]
    for column_class_name, label in class_label_dict.items():
        one_class_total_predict += total_predictions_dict[row_class_name][column_class_name]
    
    for column_class_name, label in class_label_dict.items():
        one_class_cur_predict = total_predictions_dict[row_class_name][column_class_name]
        # acc
        #acc = round((one_class_cur_predict / one_class_total_predict), 4)
        #line.append(str(acc))
        # num
        line.append(one_class_cur_predict)
    print(one_class_total_predict)
    print(str(one_class_total_predict))
    line.append(str(one_class_total_predict))
    print(line)
    csv_write.writerow(line)
out.close()